<h1> SIH2020<h1>
<h2>Day-one-activity-HunTer_SQuaD<h2>
<h4>Importing modules<h4>
Note: Modules may change or may add some more modules for final code.

In [ ]:
from pydub import AudioSegment
import IPython
import math
import noisereduce as nr
from array import *
import numpy
from scipy.io import wavfile
from scipy.io.wavfile import write
import matplotlib.pyplot as plt
import os
import sys
import speech_recognition as sr
import wave
import contextlib as cl
import re
import soundfile as sf
from pydub import AudioSegment
from pydub.playback import play 
from pydub.silence import split_on_silence
from pydub.silence import detect_silence

<h4>Performing Noise Elimination</h4>

In [ ]:
def noise_reduce(audio_data,rate,win_length,amp_adjust):
    
    import noisereduce as nr
    
    mean=0
    noise_list=[]
    mean_list=[]
    win_length=win_length
    amp_adjust=amp_adjust
    for j in range(0,len(audio_data)):
        if(j>0 and j%win_length==0):   
            mean=math.sqrt(mean/win_length)
            
            mean_list.append(mean)
            mean=0
            mean+=audio_data[j]**2
        else:
            mean+=audio_data[j]**2
    k=0
    
    for i in range(0,len(mean_list)):
        if i>0 and i<len(mean_list)-1:
            if(mean_list[i-1]<mean_list[i]+amp_adjust and mean_list[i-1]>mean_list[i]-amp_adjust):
                if(mean_list[i+1]<mean_list[i]+amp_adjust and mean_list[i+1]>mean_list[i]-amp_adjust):
                    if k==0:
                        noise_list.append((i-1)*win_length)
                        k+=1
                else:
                    if(k>0):
                        noise_list.append((i+1)*win_length)
                        k=0
                    else:
                        k=0
                        noise_list.append((i-1)*win_length)
                        noise_list.append((i+1)*win_length)
        elif(i==len(mean_list)-1):
            if(mean_list[i-1]<mean_list[i]+amp_adjust and mean_list[i-1]>mean_list[i]-amp_adjust):
                noise_list.append((i+1)*win_length)
                

    diff=0
    for i in range(0,len(noise_list)-1):
        if(i%2==0):
            if(noise_list[i+1]-noise_list[i] > diff):
                diff = noise_list[i+1]-noise_list[i]
                noise_start = noise_list[i]+win_length
                noise_stop = noise_list[i+1]-win_length
    noisy_part =audio_data[noise_start:noise_stop]
    nr_data = nr.reduce_noise(audio_clip=audio_data1, noise_clip=noisy_part,n_grad_freq=2,n_grad_time=16,n_fft=2048,win_length=2048,hop_length=512,n_std_thresh=1.5,prop_decrease=1.0, verbose=True,)
    fig, ax = plt.subplots(figsize=(20,4))
    ax.plot(audio_data)
    fig, ax = plt.subplots(figsize=(20,4))
    ax.plot(nr_data)

In [ ]:
input_audio=input("enter the name of the audio file:")
audio_data,rate = sf.read(input_audio)
dur=(len(audio_data)+1)/float(rate)
if(dur<15):
    exit()
if(audio_data.ndim > 1):
    audio_data=np.delete(audio_data,slice(1),1)
    audio_data=np.delete(audio_data,slice(1),None)
audio_data=audio_data/32768
#print(1)
chunk_silent=100
noise_reduce(audio_data,rate,win_length=5000,amp_adjust=0.005)
chunk_rate,nf_chunks,silent_ranges=segmentation(min_silence_len=300,silence_thresh=-50,chunk_silent=chunk_silent)

<h4>Recognizing the Audio<h4>